In [1]:
from google.colab import drive
import os
import pandas as pd

In [3]:
current_directory = "/data"
os.chdir(current_directory)

In [4]:
!pip install ntlk
!pip install num2words
!pip install fuzzywuzzy
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement ntlk (from versions: none)
ERROR: No matching distribution found for ntlk
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem import PorterStemmer
from nltk import word_tokenize, sent_tokenize, pos_tag, ne_chunk
from nltk import RegexpParser
from nltk import Tree

import re
import num2words
import json

In [6]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

Intialise Classes for Product Review Identification and Segmentation

In [7]:
class ProductIdentifier:
  def __init__(self):
    pass

  def remove_non_ascii(self, text):
    return re.sub(r'[^\x00-\x7F]+',' ', text)

  def GetNounsWithNLTK(self, text, chunk_func=ne_chunk):
    chunked = chunk_func(pos_tag(word_tokenize(text)))
    continuous_chunk = []
    current_chunk = []

    for subtree in chunked:
      if type(subtree) == Tree:
        current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
      elif current_chunk:
        named_entity = " ".join(current_chunk)
        if named_entity not in continuous_chunk:
          continuous_chunk.append(named_entity)
          current_chunk = []
      else:
        continue

    return continuous_chunk

  def get_uppercased_word_series(self, text):
    temp_lst = []
    uppercased_word_lst = []
    word_lst = nltk.word_tokenize(text)
    for i in range(len(word_lst)):
      word = word_lst[i]
      starts_with_uppercase = word[0].isupper()
      if starts_with_uppercase:
        temp_lst.append(word)
        if (i == len(word_lst)-1):
          if len(temp_lst) >= 2:
            uppercased_word_lst.append(" ".join(temp_lst))
        continue
      else:
        if len(temp_lst) >= 2:
          uppercased_word_lst.append(" ".join(temp_lst))
        temp_lst = []
    return list(set(uppercased_word_lst))

  def Check_If_NP_Is_Known_Product(self,np, product_lexicon):
    for product in product_lexicon:
      if np == product:
        return True
    return False

  def Check_If_NP_Has_Product_Indicator(self, np, product_indicators):
    for product in product_indicators:
      similarity = fuzz.partial_ratio(np.lower(),product.lower())
      if similarity >= 80:
        return True
    return False

  def Check_If_NP_Is_Product(self, np, product_indicators, product_lexicon):
    if self.Check_If_NP_Is_Known_Product(np, product_lexicon):
      return True
    elif self.Check_If_NP_Has_Product_Indicator(np, product_indicators):
      return True
    else:
      return False

  def is_not_substring_of_another_product(self,product, product_lst):
    for comparison_product in product_lst:
      if product.lower() == comparison_product.lower():
        continue
      if product.lower() in comparison_product.lower():
        return False
    return True

  def detect_products_from_transcript(self, review_dict, product_indicators, product_lexicon):
    text = review_dict["transcript_text"]
    #sometimes nltk ne chunker misses out
    nouns_in_transcript = self.GetNounsWithNLTK(text)
    #parse for a series of uppercase
    uppercased_series_lst = self.get_uppercased_word_series(text)

    #nouns_in_transcript.extend(np_lst)
    nouns_in_transcript.extend(uppercased_series_lst)

    #only keep if len > 2 
    nouns_in_transcript = list(filter(lambda noun_phrase: len(nltk.word_tokenize(noun_phrase))>=2, nouns_in_transcript))

    #keep if match with brand name or contain a product term
    nouns_in_transcript = list(filter(lambda noun_phrase: self.Check_If_NP_Is_Product(noun_phrase, product_indicators, product_lexicon), nouns_in_transcript))

    #filter the products
    nouns_in_transcript = list(filter(lambda product: self.is_not_substring_of_another_product(product, nouns_in_transcript), nouns_in_transcript))

    return list(set(nouns_in_transcript))


  def contains_url(self,text):
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex, text)
    if url != []:
      return True
    else:
      return False

  def remove_url(self,text):
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    removed_url_text = re.sub(regex, "", text)
    return removed_url_text

  def clean_desc(self,text):
    text = text.replace(":", "")
    text = text.replace("*", "")
    return text

  def get_products_from_desc(self,review_dict):
    description = review_dict["description"]
    channel_title = review_dict["channel_title"]
    split_desc = description.split("\n")
    desc_sentence_with_url = list(filter(lambda txt: self.contains_url(txt), split_desc))
    #set higher threshold because even if e.g. "mal" "ibu" match seperately -- it raises the match score
    desc_sentence_without_channel_url = list(filter(lambda txt: fuzz.partial_ratio(txt.lower(),channel_title.lower()) < 80, desc_sentence_with_url))
    products = list(filter(lambda txt: self.Check_If_NP_Is_Product(txt, product_indicators, product_lexicon),desc_sentence_without_channel_url))
    products = list(filter(lambda txt: self.is_not_substring_of_another_product(txt,products),products))
    filtered_desc_product_lst = [self.clean_desc(self.remove_url(txt)) for txt in products]
    return filtered_desc_product_lst
  
  def detect_products(self,review_dict, product_indicators, product_lexicon):
    filtered_transcript_product_lst = self.detect_products_from_transcript(review_dict, product_indicators, product_lexicon)
    filtered_desc_product_lst = self.get_products_from_desc(review_dict)
    filtered_transcript_product_lst.extend(filtered_desc_product_lst)
    filtered_product_lst = list(filter(lambda txt: self.is_not_substring_of_another_product(txt,filtered_transcript_product_lst),filtered_transcript_product_lst))
    filtered_product_lst = list(set(filtered_product_lst))
    return filtered_product_lst

class CoreferenceResolver:
  def __init__(self):
    pass
  
  def GetCoreferenceWithSyntaxRules(self,text):
    coref = []
    words = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(words)
    chunk_name_lst = ["DT","DT Ref"]
    grammar = r"""
              DT Ref:
              {<DT><CD>}
          """
    chunkParser=nltk.RegexpParser(grammar)
    tree = chunkParser.parse(tagged)
    for subtree in tree.subtrees(filter=lambda t: t.label() in chunk_name_lst):
      myPhrase = ''
      for item in subtree.leaves():
        myPhrase += ' ' + item[0]
      coref.append(myPhrase.strip())
    coref = list(filter(lambda x: len(x.split()) > 1, coref))
    filtered_coref = list(set(coref))
    return filtered_coref

  def replace_all(self, text, dic):
    for i, j in dic.items():
        text = re.sub(r"\b%s\b" % i, j, text)
        # r"\b%s\b"% enables replacing by whole word matches only
    return text

  def GetNPWithSyntaxRules(self, text):
    nounphrases = []
    words = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(words)
    chunk_name_lst = ["NP"]
    grammar = r"""  NP:
              {<NN|NNS|NNP|NNPS>+}
          """
    chunkParser=nltk.RegexpParser(grammar)
    tree = chunkParser.parse(tagged)
    for subtree in tree.subtrees(filter=lambda t: t.label() in chunk_name_lst):
        myPhrase = ''
        for item in subtree.leaves():
            myPhrase += ' ' + item[0]
        nounphrases.append(myPhrase.strip())
    nounphrases = list(filter(lambda x: len(x.split()) > 1, nounphrases))
    return nounphrases

  def detect_matching_product_name(self, sentence, product_lst):
    np_lst = self.GetNPWithSyntaxRules(sentence)
    for np in np_lst:
      for product in product_lst:
        similarity_with_identified_products = fuzz.partial_ratio(np.lower(),product.lower())
        if similarity_with_identified_products >= 80:
          return (np, product)
    else:
      return None


  def resolve(self, sample_sentences, product_lst):
    curr_product_name = None
    resolved_combined_sentence_list = []
    for sentence in sample_sentences:
      #update current product name
      potentially_new_product_name_pair = self.detect_matching_product_name(sentence, product_lst)
      if potentially_new_product_name_pair != None and potentially_new_product_name_pair != curr_product_name:
        #must identify the product name better
        phrase = potentially_new_product_name_pair[0]
        curr_product_name = potentially_new_product_name_pair[1]
        #pos_tag sentence --> assign determiners after the product name to the product name
        substr_before_product = sentence.split(phrase,1)[0] + curr_product_name
        substr_after_product = sentence.split(phrase,1)[-1]
        replacements = {'This':curr_product_name,'this':curr_product_name,'it':curr_product_name, 'It':curr_product_name}
        substr_after_product = self.replace_all(substr_after_product, replacements)
        resolved_combined_sentence_list.append(substr_before_product + substr_after_product)
      
      elif curr_product_name != None:
        #replace all references with the curr_product_name
        replacements = {'This':curr_product_name,'this':curr_product_name,'it':curr_product_name, 'It':curr_product_name}
        sentence = self.replace_all(sentence, replacements)
        resolved_combined_sentence_list.append(sentence)
      
      else:
        resolved_combined_sentence_list.append(sentence)
    
    return resolved_combined_sentence_list

  def resolve_sentences(self,sample_sentences, product_lst):
    sample_sentences = [sentence.replace("\n", " ") for sentence in sample_sentences]
    resolved_list = self.resolve(sample_sentences, product_lst)
    return resolved_list

class TranscriptSegmenter:
  def __init__(self):
    self.product_identifier = ProductIdentifier()
    self.coreference_resolver = CoreferenceResolver()

  def get_sentence_list_from_transcript(self, transcript):
    phrases = nltk.sent_tokenize(transcript)
    pos_tagged = [pos_tag(nltk.word_tokenize(sentence.lower())) for sentence in phrases]
    new_combined_sentence_list = []
    for i in range(len(phrases)):
      tagged_phrase = pos_tagged[i]
      tag_of_first_word = tagged_phrase[0][1]
      conjunctions_lst = ["IN", "CC", "WDT", "TO", "WRB", "VBG", "VBZ", "RB"]
      end_lst = ["IN", "CC", "WDT", "TO", "WRB", "VBG", "VBZ", "DT", "RB"]
      join_sentence = False
      if i >= 1:
        last_tag_in_prev_sentence = pos_tagged[i-1][-1][1]
        if last_tag_in_prev_sentence in end_lst:
          join_sentence = True
      if tag_of_first_word in conjunctions_lst:
        join_sentence = True

      if join_sentence:
        #lowercase the first letter to reduce noise when doing NP identification e.g. "Which is so beautiful" --> Which to which
        sub_clause = phrases[i].replace(phrases[i][0],phrases[i][0].lower(),1)
        new_combined_sentence_list[-1] = new_combined_sentence_list[-1] + " " + sub_clause
      else:
        new_combined_sentence_list.append(phrases[i])
    return new_combined_sentence_list

  def segment_transcript(self, filtered_product_lst, resolved_list, new_combined_sentence_list):
    product_review_dict = {}
    for i in range(len(resolved_list)):
      resolved_sentence = resolved_list[i]
      for product in filtered_product_lst:
        if product in resolved_sentence:
          if product in product_review_dict:
            product_review_dict[product]["resolved"] = product_review_dict[product]["resolved"] + resolved_sentence
            product_review_dict[product]["original"] = product_review_dict[product]["original"] + new_combined_sentence_list[i]
          else:
            product_review_dict[product] = {"resolved":resolved_sentence, "original":new_combined_sentence_list[i]}
    
    return product_review_dict

  def split_transcript_by_detected_products(self, review_dict, product_indicators, product_lexicon):
    transcript = review_dict["transcript_text"]
    filtered_product_lst = self.product_identifier.detect_products(review_dict, product_indicators, product_lexicon)
    sample_sentences = self.get_sentence_list_from_transcript(transcript)
    resolved_lst = self.coreference_resolver.resolve_sentences(sample_sentences, filtered_product_lst)
    product_review_dict = self.segment_transcript(filtered_product_lst, resolved_lst, sample_sentences)
    return product_review_dict

  #Case of video having chapters

  def process_raw_transcript(self, raw_transcript):
    text_list = [segment["text"] for segment in raw_transcript]
    text = " ".join(text_list)
    text = re.sub("\n", " ", text)
    #remove non-ascii characters -- to remove foreign lang characters
    text = self.product_identifier.remove_non_ascii(text)
    return text

  def split_transcript_by_chapters(self, raw_transcript, chapters):
    starts = [round(segment["start"]) for segment in raw_transcript]
    product_review_dict = {}
    chapter_index_list = []
    for chapter in chapters:
      time_in_sec = round(chapter["time_in_min"] * 60)
      #get a index list of chapters 
      new_insertion_index = bisect.bisect_left(starts, time_in_sec)
      chapter_index_list.append(new_insertion_index)
    chapter_index_list.append(len(raw_transcript)-1)
    for i in range(len(chapter_index_list)-1):
      title = chapters[i]["title"]
      start_index = chapter_index_list[i]
      end_index = chapter_index_list[i+1]
      original_raw_transcript = self.process_raw_transcript(raw_transcript[start_index:end_index])
      sample_sentences = [segment["text"] for segment in raw_transcript[start_index:end_index]]
      resolved_raw_transcript = self.coreference_resolver.resolve_sentences(sample_sentences, [title])
      product_review_dict[title] = {"resolved":resolved_raw_transcript, "original":original_raw_transcript}
    return product_review_dict


  def get_product_review_dict(self, review_dict, product_indicator, product_lexicon):
    chapters = sample_review_dict["chapters"]
    product_review_dict = {}
    if chapters == []:
      product_review_dict = self.split_transcript_by_detected_products(review_dict, product_indicator, product_lexicon)
    else:
      raw_transcript = sample_review_dict["transcript_raw"]
      product_review_dict = self.split_transcript_by_chapters(raw_transcript, chapters)
    return product_review_dict


Load Youtube Data

In [8]:
import bisect
import json
from fuzzywuzzy import fuzz

In [9]:
#Load data scraped using various Youtube API
with open('youtube_data.json', 'r') as f:
  reviews = json.load(f)

Demo

In [10]:
product_indicators = pd.read_csv("Category Lexicon.csv")["Category"].to_list()
product_lexicon = pd.read_csv("Product Lexicon.csv")["Product"].to_list()

In [11]:
transcript_segmenter = TranscriptSegmenter()

In [12]:
sample_review_id = "8qvjvbYvYsA"
sample_review_dict = reviews[sample_review_id]
product_review_dict = transcript_segmenter.get_product_review_dict(sample_review_dict, product_indicators, product_lexicon)
for k, v in product_review_dict.items():
  print(k)
  print(v["original"].replace("\n", " "))

Happy New Year
- Hello, my lovelies, and Happy New Year.Today, it's all about the best affordable makeup of 2021 from my rendition, my opinion, my makeup that I just kept going towards, kept reaching towards, and I just couldn't get enough of.This right here is by e.l.f.I don't usually like primers, but this one I actually (laughs) really, really enjoyed.
Primer elf Oil Control Lotion 
This is their Oil Control Liquid Lotion with tea tree oil.This might put you off if you have dry skin, but I found that this just helped balance the oil throughout the day, especially in a humid, warm, hot climate, definitely towards the spring and summertime, but I really enjoyed this throughout the entire year, and I would mostly focus in areas where I would get extra shiny, like right here, the T-zone, or just even use it all over my face if I knew it was gonna be really hot and humid that day. and it also smells really nice. but it doesn't really mattify you.It just gives a really nice, clean base to

In [13]:
sample_review_id = "wheUpE7tpKA"
sample_review_dict = reviews[sample_review_id]
product_review_dict = transcript_segmenter.get_product_review_dict(sample_review_dict, product_indicators, product_lexicon)
for k, v in product_review_dict.items():
  print(k)
  print(v["original"].replace("\n", " "))

CLEANSING BALM
but does not leave a greasy residue. So I put, I have makeup on right now, so I'm going to put it all over my face and my lips. I'm not scared of it, even on my eyes. And then I will take a little bio-friendly pad, 'cause I don't use the cotton pads anymore, and warm water. I put it on the pad, hold it between my fingers like this and do the first layer of removal, and you will see how it takes it off immediately. I used to use very, very hot water. I have a tendency to have more dry skin than oily skin, so that's what I love about this cleanser is it really keeps your face moisturized. And I usually finish with just a regular wash cloth that I wet. And my mother always used to say, moisturizer and cleanser, keeping your face clean. She used to say soap and water, and so I used to use bubbly soap because I thought that that squeaky, dry skin feeling meant that my face was very clean. But because of this, if you notice, just warm water literally took off all the makeup an

Process of segmenting the text transcript
- For videos with chapters, we will use the chapters to segment the transcript. This process below is the algorithm for videos without chapters

In [14]:
sample_review_id = "8qvjvbYvYsA"
sample_review_dict = reviews[sample_review_id]
transcript = sample_review_dict["transcript_text"]

Change from Iteration 2: Parse for uppercased phrases e.g. "True Botanical Cleansing Serum" due to loopholes when filtering with product indicators and the variety of makeup product names (not just series of noun)

In [15]:
product_identifier = ProductIdentifier()
#Get products from transcript and description using syntax parsing rules
filtered_product_lst = product_identifier.detect_products(sample_review_dict, product_indicators, product_lexicon)
filtered_product_lst

['Maxxx Nourishing Lip Plumper',
 'Mascara Primer L’Oreal Voluminous Primer ',
 'CC Cream',
 'Lip Liner NYX Lip Pencil (Nude Truffle) ',
 'Bronzer Juvia’s Place Bronzer Palette (Light) ',
 'Eyebrow Winky Lux Micro Universal Brow Pencil ',
 'Mascara Maybelline Lash Sensational ',
 'Concealer NYX Serum Concealer (Vanilla) ',
 'Primer elf Oil Control Lotion ',
 'Me Concealer Serum',
 'Lip Gloss Milani Keep it Full Maxxx Lip Plumper (110 Little Secret) ',
 'Maybelline Age Rewind Concealer',
 "L'Oreal Voluminous Primer",
 'Foundation L’Oreal Skin Serum (0.5-2 Very Light) ',
 'The NYX Bear',
 "L'Oreal True Match Hyaluronic Tinted Serum",
 'Lip Liner Milani Color Statement Lip Liner (04 All Natural) ',
 'Happy New Year',
 'Cream Blush Elf Putty Blush (Tahiti) ',
 'Highlighter Catrice More Than Glow Highlighter (Supreme Rose Beam) ',
 'Gel Eyeliner Milani Stay Put Eyeliner ',
 'Milani Blush',
 'Eyebrow Gel NYX The Brow Glue ',
 'Eyeshadow Primer Wet n Wild PhotoFocus Base  ',
 'Eyeshadow elf B

Same as Iteration 2: Manually join and process the sub-sentences in the transcript

In [16]:
#Do some manual joining of transcript sub-sentences as the transcript splits by the captions so 
#e.g. "This is a wonderful primer" --> "This is", "A wonderful primer"
transcript_segmenter = TranscriptSegmenter()
sample_sentences = transcript_segmenter.get_sentence_list_from_transcript(transcript)
sample_sentences

['- Hello, my lovelies, and Happy New Year.',
 "Today, it's all about the\nbest affordable makeup of 2021\nfrom my rendition, my opinion, my makeup\nthat I just kept going\ntowards, kept reaching towards,\nand I just couldn't get enough of.",
 'Some of them are newer products,\neither the beginning of the year, mid,\nor even towards the end of the year,\nand then some of them are\njust rediscovered loves.',
 "Let's begin, starting\nfirst with the primer.",
 'This right here is by e.l.f.',
 "I don't usually like primers,\nbut this one I actually\n(laughs) really, really enjoyed.",
 'This is their Oil Control\nLiquid Lotion with tea tree oil.',
 "This might put you off\nif you have dry skin,\nbut I found that this just helped balance\nthe oil throughout the\nday, especially in a humid,\nwarm, hot climate,\ndefinitely towards the\nspring and summertime,\nbut I really enjoyed this\nthroughout the entire year,\nand I would mostly focus in areas\nwhere I would get extra shiny,\nlike right he

Change from Iteration 2: Add manual coreference resolution due to weaknesses in model based coreference as we are looking to specificially resolve only for product names
- Existing models tend to be too slow or cannot handle large text chunks (e.g. AllenNLP)
- There are fast models like fastcoref package but they do not identify the product name to resolve 
- e.g. might change "this" to "this pencil" instead of "Nyx Eyebrow Pencil"

In [17]:
#Conduct coreference resolution to better differentiate the sentences in the transcript
coreference_resolver = CoreferenceResolver()
resolved_lst = coreference_resolver.resolve_sentences(sample_sentences, filtered_product_lst)
resolved_lst

['- Hello, my lovelies, and Happy New Year.',
 "Today, Happy New Year's all about the best affordable makeup of 2021 from my rendition, my opinion, my makeup that I just kept going towards, kept reaching towards, and I just couldn't get enough of.",
 'Some of them are newer products, either the beginning of the year, mid, or even towards the end of the year, and then some of them are just rediscovered loves.',
 "Let's begin, starting first with the primer.",
 'Happy New Year right here is by e.l.f.',
 "I don't usually like primers, but Happy New Year one I actually (laughs) really, really enjoyed.",
 'This is their Primer elf Oil Control Lotion  with tea tree oil.',
 "Primer elf Oil Control Lotion  might put you off if you have dry skin, but I found that Primer elf Oil Control Lotion  just helped balance the oil throughout the day, especially in a humid, warm, hot climate, definitely towards the spring and summertime, but I really enjoyed Primer elf Oil Control Lotion  throughout the e

Change from Iteration 2: Directly join sentences that mention the product name together as chunking is not very accurate as number of chunks increases. Usually in a video, they are also doing a makeup tutorial along with the review so this confuses the chunking process

In [18]:
#For each product, find the sentences that contain mention of the product and join them together to form a segment

product_review_dict = transcript_segmenter.segment_transcript(filtered_product_lst, resolved_lst, sample_sentences)
for k, v in product_review_dict.items():
  original_chunk = v["original"].replace("\n", " ")
  print(f"Product name: {k}")
  print(f"Associated Chunk: {original_chunk}")

Product name: Happy New Year
Associated Chunk: - Hello, my lovelies, and Happy New Year.Today, it's all about the best affordable makeup of 2021 from my rendition, my opinion, my makeup that I just kept going towards, kept reaching towards, and I just couldn't get enough of.This right here is by e.l.f.I don't usually like primers, but this one I actually (laughs) really, really enjoyed.
Product name: Primer elf Oil Control Lotion 
Associated Chunk: This is their Oil Control Liquid Lotion with tea tree oil.This might put you off if you have dry skin, but I found that this just helped balance the oil throughout the day, especially in a humid, warm, hot climate, definitely towards the spring and summertime, but I really enjoyed this throughout the entire year, and I would mostly focus in areas where I would get extra shiny, like right here, the T-zone, or just even use it all over my face if I knew it was gonna be really hot and humid that day. and it also smells really nice. but it doesn